<a href="https://colab.research.google.com/github/hhicks13/portfolio-optimization/blob/main/Lagrange_Vault.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


\begin{array}{ll} \mbox{max} & y^T\mathbf{u} - \frac{1}{2} \gamma y^T M y\\
\mbox{subject to} & {\bf \log y}^Tp = -F,
\end{array}
$\textbf{Solution:}$
> Define the Lagrangian:
$$\mathcal{L}(y,\lambda) = y^T\mathbf{u} - \frac{1}{2}\gamma y^T M y - \lambda(y^Tp + F) $$
> Derive 1st order conditions:
$$\begin{align} \frac{\partial\mathcal{L}}{\partial y} &=& \mathbf{u} -\gamma M y - \gamma p  \overset{\Delta}{=} 0 \\
 \frac{\partial\mathcal{L}}{\partial \lambda} &=& F + y^T {\bf p} \overset{\Delta}{=} 0
 \end{align}$$


# model simulator for C-value #

In [ ]:

import numpy as np

# Underwriter tokens are minted / burned on deposit / withdrawal.
# Buyer tokens (options) are minted / burned on purchase / exercise.
# Assumption is tokens are 1 = 1 trading unit

class OptionsPool:
    underlying = 0
    underwriters_tokens = 0
    buyers_tokens = 0
    pool_state = 0
    fee_percent = 0
    min_trade = 0.0
    fees = 0
    alpha = 1

    def __init__(self, underwriters_tokens, buyers_tokens, pool_state, fee_percent, min_trade, deposit_at_nth_timestamp: int, tick: int):
        self.underwriters_tokens = underwriters_tokens
        self.buyers_tokens = buyers_tokens
        self.pool_state = pool_state
        self.fee_percent = fee_percent
        self.min_trade = min_trade
        self.c_value = 2 * np.e
        self.bSch = 0
        self.market_price = 0
        self.capital = pool_state
        self.underlying = 0
        self.deposit_at_nth_timestamp = deposit_at_nth_timestamp
        self.tick = tick
        self.batch_deposits = 0

    def update_price(self, underlying: float, bsch_price: float, market_price: float):
        self.underlying = underlying
        self.bSch = bsch_price
        self.market_price = market_price

    def update_c_value(self, x_t: float, c_t_1: float):
        self.c_value = c_t_1 * np.exp(-self.alpha * x_t)

    def slippage(self, x_t: float)  -> float:
        return (1 - (np.exp(-self.alpha * x_t))) / (self.alpha * x_t)

    def batch_deposit(self, trade: float, new_capital=True):
        if (self.tick % self.deposit_at_nth_timestamp) == 0:
            self.deposit(trade + self.batch_deposits, new_capital)
            self.batch_deposits = 0
        else:
            self.batch_deposits += trade

    # liquidity deposited
    def deposit(self, trade: float, new_capital=True):
        x_t = trade / (trade + self.pool_state)
        self.update_c_value(x_t, self.c_value)

        self.pool_state = self.pool_state + trade
        if new_capital:
            self.capital = self.capital + trade

    # liquidity withdrawn
    def withdraw(self, trade: float):
        x_t = -trade / self.pool_state
        self.update_c_value(x_t, self.c_value)

        self.pool_state = self.pool_state - trade
        self.capital = self.capital - trade

    # options purchased
    def buy(self, trade: float, apply_fee = False):
        # assert trade > self.min_trade
        if apply_fee:
            trade =  trade + trade * self.fee_percent / 100

        premia = (trade / self.underlying) * (self.c_value * self.bSch)
        x_t = (-trade) / self.pool_state

        # print(premia, trade, self.underlying, self.bSch, self.c_value)

        self.update_c_value(x_t, self.c_value)
        premia = premia * self.slippage(x_t)

        self.pool_state = self.pool_state - trade + premia
        self.capital += premia
        self.underwriters_tokens = self.underwriters_tokens + trade
        self.buyers_tokens = self.buyers_tokens + trade

        return premia

    def emulate_expiration(self, expired_amount: float):
        self.batch_deposit(expired_amount, new_capital=False)

        self.buyers_tokens = self.buyers_tokens - expired_amount
        self.underwriters_tokens = self.underwriters_tokens - expired_amount

    def emulate_execution(self, expired_amount: float, in_the_money: float):
        self.batch_deposit(expired_amount - in_the_money, new_capital=False)

        self.capital = self.capital - in_the_money
        self.buyers_tokens = self.buyers_tokens - expired_amount
        self.underwriters_tokens = self.underwriters_tokens - expired_amount

In [ ]:
#S: spot price
#K: strike price
#T: time to maturity
#r: interest rate
#q: rate of continuous dividend paying asset
#v: volatility of underlying asset

from scipy.stats import norm
def black_scholes(S, K, T, v, option = 'call'):

    d1 = (np.log(S / K) + (0.5 * v ** 2) * T) / (v * np.sqrt(T))
    d2 = d1 - (v * np.sqrt(T))

    if option == 'call':
        return S * norm.cdf(d1, 0.0, 1.0) - K * norm.cdf(d2, 0.0, 1.0)
    if option == 'put':
        return K * norm.cdf(-d2, 0.0, 1.0) - S * norm.cdf(-d1, 0.0, 1.0)

In [ ]:
import requests

import pandas as pd
from datetime import datetime, timedelta

simulation_volatility = 0.2
simulation_mean = 0.1

def emwa_uneven_var(log_returns, t_2, t_1, m_prev, var_prev, tau = 7):
    delta = (t_2 - t_1) / 60 / 60 / 24 / 1000
    l = 1 / tau
    omega = 1 - np.exp(-delta * l)
    m = omega * log_returns + (1 - omega) * m_prev
    v = (1 - omega) * (var_prev + omega * (log_returns - m_prev) ** 2)
    return m, v


URL = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range?vs_currency=usd&from=1591131600&to=1622667600"
underlying_data = requests.get(url = URL).json()['prices']

def run_simulation(market_price_coef: float, maturity: float, mode='ITM', option_kind='Call'):
    data = []
    pool = OptionsPool(
        underwriters_tokens=0,
        buyers_tokens=0,
        pool_state=10000,
        fee_percent=1,
        min_trade=1,
        deposit_at_nth_timestamp=1,
        tick=1)

    black_sch_price = 0
    realised_var = 0
    realised_mean = 0

    options_state = {}

    for i, [timestamp, underlying_price] in enumerate(underlying_data):
        rand_maturity = max(2, np.floor(np.random.normal(maturity, maturity * 0.5)))

        pool.tick = i

        if i > 0:
            yesterday_underlying_price = underlying_data[i-1][1]
            yesterday_timestamp = underlying_data[i-1][0]
        else:
            yesterday_underlying_price = underlying_price
            yesterday_timestamp = timestamp

        # ETH/USDC CALL == PUT USDC/ETH
        if option_kind == 'Put':
            underlying_price = 1 / underlying_price

        log_return = np.log(underlying_price/yesterday_underlying_price)
        realised_mean, realised_var = emwa_uneven_var(log_return, timestamp, yesterday_timestamp, realised_mean, realised_var)
        realised_vol = np.sqrt(realised_var * 365)

        if mode == 'ITM' and option_kind == 'Call':
            strike = underlying_price * 0.95 * np.random.normal(1, 0.2)
            black_sch_price = black_scholes(underlying_price, strike, maturity / 365, realised_vol)
        if mode == 'OTM' and option_kind == 'Call':
            strike = underlying_price * 1.05 * np.random.normal(1, 0.2)
            black_sch_price = black_scholes(underlying_price, strike, maturity  / 365, realised_vol)
        if mode == 'ITM' and option_kind == 'Put':
            strike =  underlying_price * 0.95 * np.random.normal(1, 0.2)
            black_sch_price = black_scholes(underlying_price, strike, maturity / 365, realised_vol)
        if mode == 'OTM' and option_kind == 'Put':
            strike = underlying_price * 1.05 * np.random.normal(1, 0.2)
            black_sch_price = black_scholes(underlying_price, strike, maturity  / 365, realised_vol)

        market_price = black_sch_price * market_price_coef
        pool.update_price(underlying=underlying_price, bsch_price=black_sch_price, market_price=market_price)

        random_trade = max(0.1, np.random.normal(loc=pool.pool_state * simulation_mean, scale=pool.pool_state * simulation_volatility))
        random_noise = np.random.uniform(low=0, high=pool.pool_state * simulation_mean)
        random_binary_dist = np.random.dirichlet(np.ones(2),size=1)[0]

        # suppliers incentivised
        if black_sch_price * pool.c_value >= market_price:
            deposit = random_trade
            withdrawal = random_noise * random_binary_dist[0]
            purchase = random_noise * random_binary_dist[1]
        # buyers incentivised
        else:
            purchase = random_trade
            withdrawal = random_noise * random_binary_dist[0]
            deposit = random_noise * random_binary_dist[1]

        premia = pool.buy(purchase)
        expired = 0
        executed = 0
        options_state[timestamp] = {'payed_per_share': strike + premia / purchase, 'underlying_amount': purchase, 'maturity': rand_maturity}
        next_state = {}

        for (ts, locked) in options_state.items():
            if datetime.fromtimestamp(ts / 1000) + timedelta(days=locked['maturity']) <= datetime.fromtimestamp(timestamp / 1000):
                # print(datetime.fromtimestamp(timestamp), datetime.fromtimestamp(ts), locked, pool.pool_state, pool.c_value)
                if locked['payed_per_share'] < underlying_price:
                    executed = locked['underlying_amount']
                    pool.emulate_execution(locked['underlying_amount'], locked['underlying_amount'] * (underlying_price - locked['payed_per_share']) / underlying_price)
                else:
                    expired = locked['underlying_amount']
                    pool.emulate_expiration(locked['underlying_amount'])
            else:
                next_state[ts] = locked

        options_state = next_state

        pool.batch_deposit(deposit)
        pool.withdraw(withdrawal)

        data.append({
            'Underlying': underlying_price,
            'Strike': strike,
            'Log returns': log_return,
            'Realised Vol': realised_vol,
            'Tick': datetime.fromtimestamp(np.floor(timestamp / 1000)),
            'Bsch Price': black_sch_price,
            'Trade': random_trade,
            'Noise': random_noise,
            'Deposit': deposit,
            'Batch deposits cumulative': pool.batch_deposits,
            'Batch deposits mean': pool.batch_deposits / pool.deposit_at_nth_timestamp,
            'Withdrawal': withdrawal,
            'Expired': expired,
            'Executed': executed,
            'Purchased': purchase,
            'Premia paid': premia,
            'Total cap': pool.capital,
            'Free cap': pool.pool_state,
            'Simulated random maturity': rand_maturity,
            'Pool Capital Utilization': 1 - (pool.pool_state / pool.capital),
            'C-value': pool.c_value,
            'Underwriter tokens': pool.underwriters_tokens,
            'Buyers tokens': pool.buyers_tokens,
            'Price difference from market': (black_sch_price * pool.c_value - market_price) / market_price
        })

    return pd.DataFrame(data)



# Model for Vault Optimization #

$\textbf{Vault Optimization via Lagrange Minimization }$ \
$\textbf{ 1 }$ Let $I_n$ be the identity matrix, and $D$ be the initial deposit.We define the following terms:
$$\begin{align}
\mathbf{u} &\overset{\Delta}{=}& n^{-1}\mathbf{1}_n \\
D &\overset{\Delta}{=}& \texttt{ initial deposit} \\
C_i &\overset{\Delta}{=}& \texttt{ pool i C value} \\
p_i &\overset{\Delta}{=}& \texttt{ pool i free liquidity} \\
w_i &\overset{\Delta}{=}& \texttt{pool i weights for deposit} \\
x_i &\overset{\Delta}{=}& e^{-\frac{w_i D}{p_i}}  \\
y_i &\overset{\Delta}{=}& \log (C_ix_i) \\
&=& \log C_i + \log x_i
\end{align}$$

$\textbf{ 2 }$Matrix Formulation of the log-normal mean estimator $y^T\mathbf{u}$: \
$$
\begin{align}
y^T\mathbf{u} &=& \overline{\log Cx} \\
&=& \frac{\sum_i \log(Cx)_i}{n}\\
\end{align}
$$ \
$\textbf{ 3 }$Matrix Formulation of the log-normal sample covariance estimator $y^TMy$: \
$$\begin{align}
M &=& {I}_n - n^{-1}\mathbf{1}_n\mathbf{1}_n^T \\
y^TMy &=& (n - 1)\sum_i^n({\log(Cx)}_i - \overline{\log (Cx)})^2 \\
\end{align}
$$ \
$\textbf{ 4 }$Due to the fact that all deposits cause a decrease in C value, and all weights must be non-negative and sum to 1, we have the following conditions that must hold:
$$\begin{align}
 -\ln x_i \frac{p_i}{D} &=& w_i \\
 \sum p_i \ln x_i + D &=& 0
 \end{align}$$ \
$\textbf{ 5 }$Proof that condition from $\mathbf{4}$: $\sum_i p_i \log x_i + D = 0$ is equivalent to condition $\sum_i p_i y_i + F = 0 $ \
$$\begin{align}
y &=& \log Cx \\
&=& \log C + \log x \\
y^Tp &=& \sum_i p_i(\log C_i + \log x_i) \\
&=& \log C^T p + \log x^T p \\
F &\overset{\Delta}{=}& D - \log C^T p\\
y^Tp + F &=& \log x^T p + D \\
\end{align}$$





In [ ]:
import cvxpy as cp
simulations_cs = []
free_cap = []
numpools = 1
try:
    for i in range(0, numpools):
         df = run_simulation(market_price_coef=5, maturity=28, mode='ITM', option_kind='Call')
         simulations_cs.append(df['C-value'])
         free_cap.append(df['Free cap'])
finally:
    print(simulations_cs[-1])
    print(free_cap[-1])

0      4.452403
1      4.691476
2      6.449879
3      6.502631
4      6.638345
         ...   
360    4.770536
361    4.456183
362    4.611577
363    5.587918
364    4.828293
Name: C-value, Length: 365, dtype: float64
0      1.273745e+04
1      1.207202e+04
2      8.280957e+03
3      8.295456e+03
4      8.227852e+03
           ...     
360    5.998725e+11
361    6.441058e+11
362    6.210493e+11
363    5.952495e+11
364    7.059193e+11
Name: Free cap, Length: 365, dtype: float64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
def F(log_c,p,D):
  return D - _log_c.T@p 

def expression(y,p,log_c,D):
  _F = F(log_c,p,D)
  return y.T@p + _F

def weights(y,p,log_c,D):
  wdp =  -1*(y - log_c)
  w = cp.multiply(wdp,p)/D 
  return w

def extract_w(y,p,log_c,D):
  wdp = -1*(y-log_c)
  w = np.multiply(wdp,p)/D
  return w

In [ ]:
# simulate
def generate_C(numpools: int):
  simulations_cs = []
  free_cap = []
  numpools = 1
  try:
    for i in range(0, numpools):
      df = run_simulation(market_price_coef=5, maturity=28, mode='ITM', option_kind='Call')
      simulations_cs.append(df['C-value'])
      free_cap.append(df['Free cap'])
  finally:
    print("C,Pn generated")
  
  labels = ['asset '+str(i) for i in range(1,numpools+1)]
  C = pd.DataFrame(simulations_cs).T
  C.columns = labels
  Pn = pd.DataFrame(free_cap).T
  Pn.columns = labels
  return C,Pn

# Case 1 #

In [ ]:
"""
case 1 
D == p
n == 1
"""

import cvxpy as cp
n = 1
C,Pn = generate_C(n)

_c = C.iloc[0,:].values
_log_c = np.log(_c)
_p = Pn.iloc[0,:].values
n = len(_c)
_M = M(n)
_u = u(n)
D = _p
y = cp.Variable(n)
gamma = cp.Parameter(nonneg=True)
mean = _u.T@y 
var = cp.quad_form(y, _M)
prob = cp.Problem(cp.Maximize(mean - gamma*var), 
                  [expression(y,_p,_log_c,D) == 0,
                  cp.sum(weights(y,_p,_log_c,D)) == 1],)
SAMPLES = 100
risk_data = np.zeros(SAMPLES)
ret_data = np.zeros(SAMPLES)
gamma_vals = np.logspace(-2, 3, num=SAMPLES)
for i in range(SAMPLES):
    gamma.value = gamma_vals[i]
    prob.solve()
    risk_data[i] = cp.sqrt(var).value
    ret_data[i] = mean.value

extract_w(y.value,_p,_log_c,D)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


C,Pn generated


array([1.])

# Case 2 #

In [ ]:
"""
case 2
n=3

p1 = 10
p2 = 10
p3 = 100

D=60

c1 =1 
c2 =1000
c3=1
"""
_c = np.array([1,1000,1])
_log_c = np.log(_c)
_p = np.array([10,10,100]) 
n = 3
_M = M(n)
_u = u(n)
D = 60
y = cp.Variable(n)
gamma = cp.Parameter(nonneg=True)
mean = _u.T@y 
var = cp.quad_form(y, _M)
prob = cp.Problem(cp.Maximize(mean - gamma*var), 
                  [expression(y,_p,_log_c,D) == 0,
                   cp.sum(weights(y,_p,_log_c,D)) == 1,
                   weights(y,_p,_log_c,D) >= 0],)

SAMPLES = 100
risk_data = np.zeros(SAMPLES)
ret_data = np.zeros(SAMPLES)
gamma_vals = np.logspace(-2, 3, num=SAMPLES)
for i in range(SAMPLES):
    gamma.value = gamma_vals[i]
    prob.solve()
    risk_data[i] = cp.sqrt(var).value
    ret_data[i] = mean.value

extract_w(y.value,_p,_log_c,D)

array([-9.86513061e-19,  1.00000000e+00, -7.08923547e-17])